In [ ]:
# Data : 
#     +) https://www.kaggle.com/datasets/aibloy/fairface
#     +) https://www.kaggle.com/datasets/jangedoo/utkface-new

In [ ]:
# ============================================================
# 1. STANDARD LIBRARIES (Thư viện chuẩn Python)
# ============================================================
import os
import math
import random
from glob import glob
from typing import Dict
from collections import Counter

# ============================================================
# 2. DATA SCIENCE & UTILITIES (Xử lý dữ liệu & Ảnh)
# ============================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.notebook import tqdm

# Scikit-learn
from sklearn.model_selection import train_test_split

# ============================================================
# 3. PYTORCH FRAMEWORK
# ============================================================
import torch
import torch.nn as nn
import torch.serialization

# Data Handling
from torch.utils.data import Dataset, DataLoader

# Mixed Precision Training (Tăng tốc & giảm VRAM)
from torch.cuda.amp import autocast, GradScaler

# Computer Vision
import torchvision.transforms as T
from torchvision import datasets  

# Data processing

In [ ]:
# # --- Load dữ liệu gốc ---
# train_df = pd.read_csv("/kaggle/input/fairface/FairFace/train_labels.csv")
# val_df   = pd.read_csv("/kaggle/input/fairface/FairFace/val_labels.csv")

# # --- Làm sạch & thêm đường dẫn ---
# for df, subset in [(train_df, "train"), (val_df, "val")]:
#     df.drop(columns=['service_test'], errors='ignore', inplace=True)
#     df['file'] = df['file'].apply(lambda f: os.path.join("/kaggle/input/fairface/FairFace", f))
#     df['file'] = df['file'].str.replace(f"{subset}/{subset}", subset)

# # --- Mapping nhãn ---
# gender_map = {'Male': 0, 'Female': 1}
# race_map = {
#     'White': 0, 'Black': 1, 'Latino_Hispanic': 2,
#     'East Asian': 3, 'Southeast Asian': 4, 'Indian': 5, 'Middle Eastern': 6
# }
# age_map = {
#     '0-2': 0, '3-9': 1, '10-19': 2, '20-29': 3,
#     '30-39': 4, '40-49': 5, '50-59': 6, '60-69': 7, 'more than 70': 8
# }

# train_df['gender'] = train_df['gender'].map(gender_map)
# val_df['gender']   = val_df['gender'].map(gender_map)
# train_df['race']   = train_df['race'].map(race_map)
# val_df['race']     = val_df['race'].map(race_map)
# train_df['age']    = train_df['age'].map(age_map)
# val_df['age']      = val_df['age'].map(age_map)

# # --- Chia riêng theo task ---
# train_gender_df = train_df[['file', 'gender']].copy()
# val_gender_df   = val_df[['file', 'gender']].copy()

# train_race_df = train_df[['file', 'race']].copy()
# val_race_df   = val_df[['file', 'race']].copy()

# train_age_df = train_df[['file', 'age']].copy()
# val_age_df   = val_df[['file', 'age']].copy()


In [ ]:
# ============================================================
# 1. UTILS (CÔNG CỤ HỖ TRỢ)
# ============================================================
def set_seed(seed=42):
    """
    Thiết lập hạt giống ngẫu nhiên (seed) cố định để đảm bảo kết quả 
    có thể tái lập (reproducible) mỗi lần chạy.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


# ============================================================
# 2. LOAD DATASET: FAIRFACE
# ============================================================
def load_fairface(base_dir: str):
    """
    Đọc dữ liệu từ bộ FairFace.
    - Input: Đường dẫn thư mục gốc.
    - Output: DataFrame train và validation đã được chuẩn hóa nhãn.
    """
    # Đọc file CSV nhãn
    train_df = pd.read_csv(os.path.join(base_dir, "train_labels.csv"))
    val_df = pd.read_csv(os.path.join(base_dir, "val_labels.csv"))

    # Bản đồ ánh xạ nhãn từ chuỗi sang số nguyên (Integer Mapping)
    gender_map = {'Male': 0, 'Female': 1}
    race_map = {
        'White': 0, 'Black': 1, 'Latino_Hispanic': 2,
        'East Asian': 3, 'Southeast Asian': 4,
        'Indian': 5, 'Middle Eastern': 6
    }
    # Chia độ tuổi thành 9 nhóm
    age_map = {
        '0-2': 0, '3-9': 1, '10-19': 2, '20-29': 3,
        '30-39': 4, '40-49': 5, '50-59': 6,
        '60-69': 7, 'more than 70': 8
    }

    # Xử lý từng tập dữ liệu (train/val)
    for df, subset in [(train_df, "train"), (val_df, "val")]:
        # Sửa lại đường dẫn file ảnh cho đúng cấu trúc thư mục
        df["file"] = df["file"].apply(
            lambda f: os.path.join(base_dir, f.replace(f"{subset}/{subset}", subset))
        )
        # Áp dụng mapping
        df["gender"] = df["gender"].map(gender_map)
        df["race"] = df["race"].map(race_map)
        df["age"] = df["age"].map(age_map)
        
        # Bỏ các dòng thiếu dữ liệu
        df.dropna(subset=["gender", "race", "age"], inplace=True)

    return train_df[["file","age","race","gender"]], \
           val_df[["file","age","race","gender"]]


# ============================================================
# 3. LOAD DATASET: UTKFACE
# ============================================================
def load_utkface(utk_dir: str):
    """
    Đọc dữ liệu từ bộ UTKFace.
    - Đặc điểm: Tên file chứa thông tin nhãn (VD: 20_1_0_timestamp.jpg -> Tuổi 20, Nam, White).
    - Cần map lại chủng tộc và nhóm tuổi để khớp với chuẩn của FairFace.
    """
    recs = []
    for f in os.listdir(utk_dir):
        if not f.lower().endswith((".jpg",".png",".jpeg")):
            continue
        
        # Phân tích tên file: Age_Gender_Race_Date
        parts = f.split("_")
        if len(parts) < 4:
            continue
        try:
            age, gender, race = int(parts[0]), int(parts[1]), int(parts[2])
        except:
            continue
            
        # Lọc nhiễu: Bỏ các tuổi quá lớn
        if age > 120:
            continue

        recs.append([os.path.join(utk_dir, f), age, gender, race])

    df = pd.DataFrame(recs, columns=["file","age_raw","gender","race"])

    # 1. Xử lý Tuổi: Chuyển tuổi cụ thể (continuous) sang 9 nhóm (bins) giống FairFace
    # Bins: 0-2, 3-9, 10-19, 20-29, ...
    bins = [2, 9, 19, 29, 39, 49, 59, 69, 200]
    df["age"] = df["age_raw"].apply(lambda a: next(i for i, b in enumerate(bins) if a <= b))

    # 2. Xử lý Race: Map từ UTK sang FairFace
    # UTK: 0:White, 1:Black, 2:Asian, 3:Indian, 4:Others
    # FairFace: 0:White, 1:Black, 3:East Asian, 5:Indian, 6:Middle Eastern
    df["race"] = (
        df["race"]
        .map({0:0, 1:1, 2:3, 3:5, 4:6}) 
        .dropna()
        .astype(int)
    )

    df = df[["file","age","race","gender"]]
    
    # Chia train/val tỉ lệ 80/20, phân tầng (stratify) theo Race để đảm bảo cân bằng
    train_df, val_df = train_test_split(
        df, test_size=0.2, random_state=42, stratify=df["race"]
    )
    return train_df.reset_index(drop=True), val_df.reset_index(drop=True)


# ============================================================
# 4. PYTORCH DATASET CLASS
# ============================================================
class MultiTaskFaceDataset(Dataset):
    """
    Dataset tùy chỉnh để load ảnh và trả về 3 nhãn cùng lúc (Tuổi, Race, Giới tính).
    """
    def __init__(self, df, transform=None):
        # Kiểm tra sự tồn tại của file ảnh, bỏ qua nếu file lỗi/không có
        exists_mask = df["file"].map(os.path.exists)
        if not exists_mask.all():
            print(f"Cảnh báo: Bỏ qua {(~exists_mask).sum()} file không tồn tại.")
            df = df[exists_mask]

        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self): 
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # Load ảnh và xử lý lỗi (nếu ảnh hỏng thì tạo ảnh đen để code không crash)
        try:
            img = Image.open(row["file"]).convert("RGB")
        except:
            img = Image.new("RGB", (112,112), (0,0,0))

        # Áp dụng Augmentation (nếu có)
        if self.transform:
            img = self.transform(img)

        # Trả về dictionary các nhãn
        labels = {
            "age":    torch.tensor(int(row["age"]), dtype=torch.long),
            "race":   torch.tensor(int(row["race"]), dtype=torch.long),
            "gender": torch.tensor(int(row["gender"]), dtype=torch.long),
        }
        return img, labels


# ============================================================
# 5. DATA BALANCING (CÂN BẰNG DỮ LIỆU)
# ============================================================
def balance_train_data(train_df, val_df):
    """
    Kỹ thuật cân bằng lại dữ liệu huấn luyện.
    Do dữ liệu khuôn mặt thường bị lệch (ví dụ: quá nhiều người Da trắng, tuổi 20-30),
    hàm này giúp lấy mẫu lại (Resampling) để các nhóm (Tuổi + Race + Gender) đồng đều hơn.
    """
    print("Đang gộp train + val để tính toán lại phân phối...")
    combined = pd.concat([train_df, val_df], ignore_index=True)

    # Tạo tổ hợp key duy nhất: (Tuổi, Race, Gender)
    combined["combo"] = list(zip(combined["age"], combined["race"], combined["gender"]))
    group_counts = combined["combo"].value_counts()

    # Tính số lượng mẫu mục tiêu (trung bình giữa nhóm ít nhất và nhiều nhất)
    min_c, max_c = group_counts.min(), group_counts.max()
    target = int((min_c + max_c) / 2)

    print(f"⚖️ Mục tiêu mỗi nhóm (Target per combo): {target}")
    balanced_parts = []

    for combo, count in group_counts.items():
        subset = combined[combined["combo"]==combo]
        if count > target:
            # Downsampling: Nếu nhiều hơn target -> Lấy ngẫu nhiên đúng bằng target
            subset = subset.sample(target, random_state=42)
        else:
            # Upsampling: Nếu ít hơn target -> Nhân bản (replace=True) lên cho đủ target
            subset = subset.sample(target, replace=True, random_state=42)
        balanced_parts.append(subset)

    balanced = pd.concat(balanced_parts).reset_index(drop=True)
    print(f"Tổng số mẫu sau khi cân bằng: {len(balanced):,}")

    # Chia lại train/val sau khi đã cân bằng (Stratify theo Race)
    train_df, val_df = train_test_split(
        balanced,
        test_size=0.1,
        random_state=42,
        stratify=balanced["race"]
    )

    return train_df.drop(columns=["combo"]), val_df.drop(columns=["combo"])

# ============================================================
# 6. MAIN EXECUTION
# ============================================================
if __name__ == "__main__":
    set_seed(42)

    # Cấu hình đường dẫn
    FAIRFACE_DIR = "/kaggle/input/fairface/FairFace"
    UTK_DIR = "/kaggle/input/utkface-new/UTKFace"

    # 1. Load dữ liệu thô
    print("--> Đang load FairFace...")
    ff_train, ff_val = load_fairface(FAIRFACE_DIR)
    print("--> Đang load UTKFace...")
    utk_train, utk_val = load_utkface(UTK_DIR)

    # 2. Gộp dữ liệu từ 2 nguồn
    train_df = pd.concat([ff_train, utk_train], ignore_index=True)
    val_df = pd.concat([ff_val, utk_val], ignore_index=True)

    # 3. Cân bằng dữ liệu (Quan trọng để tránh bias)
    train_df, val_df = balance_train_data(train_df, val_df)

    print("\nPhân bố dữ liệu sau khi xử lý (Train):")
    for col in ["age","race","gender"]:
        print(f" - {col}:", train_df[col].value_counts().sort_index().to_dict())

    # ======================================================
    # 4. Cấu hình Image Transforms (Augmentation)
    # ======================================================
    IMAGE_SIZE = 112

    # Transform cho tập Train: Tăng cường dữ liệu mạnh để chống overfitting
    train_tf = T.Compose([
        T.RandomResizedCrop(IMAGE_SIZE, scale=(0.5, 1.0), ratio=(0.9, 1.1)), # Cắt ngẫu nhiên
        T.RandomHorizontalFlip(p=0.5),                                       # Lật ngang
        T.RandomApply([T.ColorJitter(0.8, 0.8, 0.8, 0.2)], p=0.8),           # Thay đổi màu sắc/độ sáng
        T.RandomGrayscale(p=0.2),                                            # Chuyển xám ngẫu nhiên
        T.RandomApply([T.GaussianBlur(kernel_size=3, sigma=(0.1, 2))], p=0.5), # Làm mờ
        T.RandomApply([T.RandomRotation(15)], p=0.3),                        # Xoay nhẹ
        T.ToTensor(),
        T.Normalize([0.5], [0.5]) # Chuẩn hóa về [-1, 1]
    ])

    # Transform cho tập Val: Chỉ resize và chuẩn hóa
    val_tf = T.Compose([
        T.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        T.ToTensor(),
        T.Normalize([0.5], [0.5]),
    ])

    # ======================================================
    # 5. Tạo DataLoader
    # ======================================================
    train_ds = MultiTaskFaceDataset(train_df, train_tf)
    val_ds   = MultiTaskFaceDataset(val_df, val_tf)

    train_loader = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=4)
    val_loader   = DataLoader(val_ds, batch_size=128, shuffle=False, num_workers=2)

    print(f"\nSẵn sàng huấn luyện: {len(train_ds):,} mẫu train | {len(val_ds):,} mẫu val")

In [ ]:
def plot_label_distributions(df, name="Train"):
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    for i, col in enumerate(["age", "race", "gender"]):
        axes[i].hist(df[col], bins=len(df[col].unique()), rwidth=0.9, color='skyblue', edgecolor='black')
        axes[i].set_title(f"{name} {col} distribution")
        axes[i].set_xlabel(col)
        axes[i].set_ylabel("count")
    plt.tight_layout()
    plt.show()

plot_label_distributions(train_df, "Train")
plot_label_distributions(val_df, "Validation")


# Model structure

In [ ]:
# ============================================================
# 1. BASIC BLOCKS
# ============================================================

class ConvBNReLU(nn.Module):
    """
    Khối cơ bản: Convolution -> BatchNorm -> ReLU.
    Dùng để giảm chiều dữ liệu hoặc xử lý đặc trưng sơ cấp.
    """
    def __init__(self, in_ch, out_ch, kernel=3, stride=1, padding=1):
        super().__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, kernel, stride=stride, padding=padding, bias=False)
        self.bn = nn.BatchNorm2d(out_ch)
        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))


class SEBlock(nn.Module):
    """
    Squeeze-and-Excitation Block.
    Mục đích: Giúp mô hình học được "kênh nào quan trọng hơn" (Channel Attention).
    """
    
    def __init__(self, ch, reduction=16):
        super().__init__()
        hidden = max(8, ch // reduction)
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),            # Squeeze: Nén không gian (Global Info)
            nn.Conv2d(ch, hidden, 1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(hidden, ch, 1, bias=False),
            nn.Sigmoid()                        # Excitation: Tạo trọng số [0, 1]
        )

    def forward(self, x):
        return x * self.se(x) # Re-calibrate feature maps


class InvertedResidual(nn.Module):
    """
    Khối Inverted Residual (MobileNetV2).
    Cấu trúc: Expand (1x1) -> Depthwise (3x3) -> Squeeze (1x1) -> SE Block (Optional).
    Giúp tiết kiệm tham số nhưng vẫn giữ được thông tin phong phú nhờ Expand layer.
    """
    
    def __init__(self, in_ch, out_ch, stride=1, expand_ratio=6, use_se=True):
        super().__init__()
        hidden = in_ch * expand_ratio
        self.use_res = (stride == 1 and in_ch == out_ch) # Chỉ cộng residual khi kích thước không đổi
        layers = []
        
        # 1. Pointwise Convolution (Expand)
        if expand_ratio != 1:
            layers += [
                nn.Conv2d(in_ch, hidden, 1, bias=False),
                nn.BatchNorm2d(hidden),
                nn.ReLU6(inplace=True)
            ]
            
        # 2. Depthwise Convolution (Spatial context)
        layers += [
            nn.Conv2d(hidden, hidden, 3, stride, padding=1, groups=hidden, bias=False),
            nn.BatchNorm2d(hidden),
            nn.ReLU6(inplace=True)
        ]
        
        self.conv = nn.Sequential(*layers)
        
        # 3. Squeeze-and-Excitation (Attention)
        self.se = SEBlock(hidden) if use_se else nn.Identity()
        
        # 4. Pointwise Convolution (Project back to low-dim)
        self.project = nn.Sequential(
            nn.Conv2d(hidden, out_ch, 1, bias=False),
            nn.BatchNorm2d(out_ch)
        )

    def forward(self, x):
        out = self.project(self.se(self.conv(x)))
        return out + x if self.use_res else out


# ============================================================
# 2. MULTI-TASK MODEL
# ============================================================

class MultiTaskFaceModel(nn.Module):
    """
    Mô hình nhận diện khuôn mặt đa nhiệm: Tuổi, Giới tính, Chủng tộc.
    Đặc điểm:
    - Backbone ~3.5M tham số (MobileNetV2 variants).
    - Asymmetric Heads: Các nhánh đầu ra có kích thước khác nhau tùy độ khó của task.
    - Uncertainty Loss: Tự động cân bằng loss giữa các task.
    """
    
    def __init__(self, dropout: float = 0.4):
        super().__init__()
        
        # Width multiplier: Tăng độ rộng mạng lên 1.3 lần để đạt dung lượng ~7.5M params
        width_mult = 1.3 
        def C(v): return max(16, int(v * width_mult))

        # --- A. BACKBONE (Trục xương sống - Trích xuất đặc trưng) ---
        
        # Stem: Xử lý ảnh đầu vào
        self.stem = nn.Sequential(
            ConvBNReLU(3, C(32), stride=2),
            ConvBNReLU(C(32), C(48)),
        )

        # Stage 1: Đặc trưng cấp thấp (Low level features - Cạnh, góc)
        self.stage1 = nn.Sequential(
            InvertedResidual(C(48), C(64), stride=2, expand_ratio=4),
            InvertedResidual(C(64), C(64), expand_ratio=4),
            InvertedResidual(C(64), C(64), expand_ratio=4),
        )

        # Stage 2: Đặc trưng cấp trung (Mid level features - Mắt, mũi, miệng)
        self.stage2 = nn.Sequential(
            InvertedResidual(C(64), C(128), stride=2, expand_ratio=6),
            InvertedResidual(C(128), C(128), expand_ratio=6),
            InvertedResidual(C(128), C(128), expand_ratio=6),
            InvertedResidual(C(128), C(128), expand_ratio=6),
        )

        # Stage 3: Đặc trưng cấp cao (Semantic features - Lão hóa, cấu trúc mặt)
        # Tăng expand_ratio để bắt chi tiết tốt hơn cho bài toán Age/Race
        self.stage3 = nn.Sequential(
            InvertedResidual(C(128), C(192), stride=2, expand_ratio=6),
            InvertedResidual(C(192), C(256), expand_ratio=6),
            InvertedResidual(C(256), C(256), expand_ratio=6), 
        )

        # Final Pointwise & Global Pooling
        self.final_pw = ConvBNReLU(C(256), C(320), kernel=1, padding=0)
        
        # Kết hợp AvgPool (tổng quan) và MaxPool (đặc trưng nổi bật nhất)
        self.pool = nn.ModuleList([
            nn.AdaptiveAvgPool2d(1),
            nn.AdaptiveMaxPool2d(1)
        ])
        
        # Kích thước vector đặc trưng cuối cùng: 320 * width_mult * 2
        feat_dim = C(320) * 2 

        # --- B. ASYMMETRIC HEADS (Các nhánh đầu ra bất đối xứng) ---
        # Nguyên lý: Task khó cần mạng sâu/rộng hơn. Task dễ dùng mạng nhỏ hơn.
        
        # 1. GENDER HEAD (Priority: Low - Dễ nhất)
        # Binary classification -> Mạng nông.
        self.gender_head = nn.Sequential(
            nn.Linear(feat_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(128, 2)
        )

        # 2. RACE HEAD (Priority: Medium)
        # 7 classes -> Mạng vừa phải.
        self.race_head = nn.Sequential(
            nn.Linear(feat_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 7)
        )

        # 3. AGE HEAD (Priority: High - Khó nhất)
        # Age Estimation cần phân biệt các nếp nhăn/kết cấu nhỏ -> Cần nhiều tham số nhất.
        self.age_head = nn.Sequential(
            nn.Linear(feat_dim, 1024),  # Mở rộng chiều (Wide)
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            
            nn.Linear(1024, 512),       # Xử lý sâu (Deep)
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            
            nn.Linear(512, 9)           # 9 nhóm tuổi
        )

        # --- C. UNCERTAINTY WEIGHTS (Trọng số học được) ---
        # Thay vì gán weight cứng (vd: 1.0, 0.5), để model tự học độ khó của từng task.
        # log_var càng lớn -> loss task đó càng bị giảm đi.
        self.log_var_g = nn.Parameter(torch.tensor(0.0))
        self.log_var_r = nn.Parameter(torch.tensor(0.0))
        self.log_var_a = nn.Parameter(torch.tensor(0.0))

        self._init_weights()

    def forward(self, x):
        # Forward qua Backbone
        x = self.stem(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.final_pw(x)
        
        # Global Pooling & Flatten
        feats = torch.cat([p(x).flatten(1) for p in self.pool], dim=1)
        
        # Trả về dictionary kết quả
        return {
            "gender": self.gender_head(feats),
            "race": self.race_head(feats),
            "age": self.age_head(feats)
        }

    def compute_loss(self, outputs: Dict[str, torch.Tensor], labels: Dict[str, torch.Tensor]):
        """
        Tính toán Multi-task Loss sử dụng Uncertainty Weighting.
        Formula: Loss = (1 / 2*sigma^2) * Task_Loss + log(sigma)
        """
        # Label Smoothing giúp model bớt tự tin thái quá, tăng khả năng tổng quát hóa
        ce_loss = nn.CrossEntropyLoss(label_smoothing=0.1)
        ce_loss_g = nn.CrossEntropyLoss() # Gender là binary, ít nhiễu nên không cần smoothing nhiều

        # Tính loss từng phần
        lg = ce_loss_g(outputs["gender"], labels["gender"])
        lr = ce_loss(outputs["race"], labels["race"])
        la = ce_loss(outputs["age"], labels["age"])

        # Tính trọng số động (Precision = 1 / variance)
        prec_g = torch.exp(-self.log_var_g)
        prec_r = torch.exp(-self.log_var_r)
        prec_a = torch.exp(-self.log_var_a)

        # Tổng hợp loss
        loss = (
            prec_g * lg + 0.5 * self.log_var_g +
            prec_r * lr + 0.5 * self.log_var_r +
            prec_a * la + 0.5 * self.log_var_a
        )

        return loss, {
            "total": loss.item(),
            "gender_loss": lg.item(),
            "race_loss": lr.item(),
            "age_loss": la.item(),
            # Sigma (độ lệch chuẩn) càng cao nghĩa là task đó model đang thấy "khó/không chắc chắn"
            "sigma_g": self.log_var_g.exp().item(), 
            "sigma_r": self.log_var_r.exp().item(),
            "sigma_a": self.log_var_a.exp().item(),
        }

    def _init_weights(self):
        """Khởi tạo trọng số chuẩn Kaiming/Truncated Normal"""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.BatchNorm1d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.trunc_normal_(m.weight, std=0.02)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

# ============================================================
# 3. KIỂM TRA
# ============================================================
if __name__ == "__main__":
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = MultiTaskFaceModel().to(device)
    
    # 1. Kiểm tra số lượng tham số
    total_params = sum(p.numel() for p in model.parameters())
    backbone_params = sum(p.numel() for n, p in model.named_parameters() if "head" not in n)
    age_params = sum(p.numel() for p in model.age_head.parameters())
    
    print(f"Total Params: {total_params / 1e6:.2f}M")
    print("-" * 40)
    print(f"Backbone (Shared): {backbone_params / 1e6:.2f}M")
    print(f"Age Head (Heavy):  {age_params / 1e6:.2f}M")
    
    # 2. Kiểm tra luồng dữ liệu (Forward pass)
    x = torch.randn(2, 3, 112, 112).to(device) # Batch size 2, ảnh 112x112
    with torch.no_grad():
        out = model(x)
    
    print("-" * 40)
    print("Output shapes:")
    for k, v in out.items():
        print(f"  {k}: {v.shape}")

# Train - Evaluate

In [ ]:
# ============================================================
# 1. KHẮC PHỤC CÁC LỖI TƯƠNG THÍCH (COMPATIBILITY FIXES)
# ============================================================

# --- GradScaler ---
# GradScaler giúp huấn luyện Mixed Precision (FP16) để giảm VRAM và tăng tốc độ.
use_cuda = torch.cuda.is_available()
scaler = GradScaler(enabled=use_cuda)

# --- torch.load với weights_only ---
# Đăng ký các kiểu dữ liệu an toàn của Numpy để tránh lỗi bảo mật khi load checkpoint cũ.
torch.serialization.add_safe_globals([
    np.core.multiarray.scalar, np.dtype,
    np.int64, np.int32, np.int16, np.int8,
    np.uint8, np.float32, np.float64
])

def safe_load(path):
    return torch.load(path, map_location=device, weights_only=False)

# ============================================================
# 2. THIẾT LẬP MÔI TRƯỜNG (SETUP)
# ============================================================

device = torch.device("cuda" if use_cuda else "cpu")

def set_seed(seed=42):
    """
    Cố định seed để đảm bảo kết quả có thể tái lập (reproducible).
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # Tăng tốc độ nếu kích thước mạng không đổi
    torch.backends.cudnn.benchmark = True

# ============================================================
# 3. HÀM CHẠY MỘT EPOCH (RUN EPOCH)
# ============================================================

def run_epoch(model, loader, optimizer, scheduler, scaler, phase="train"):
    """
    Chạy 1 vòng lặp qua toàn bộ dữ liệu (train hoặc val).
    """
    is_train = (phase == "train")
    
    # Chuyển chế độ model: train (có dropout/BN update) hoặc eval (đóng băng)
    model.train() if is_train else model.eval()
    
    total_loss = 0.0
    # Dictionary lưu số mẫu dự đoán đúng cho từng task
    correct = {"age": 0, "race": 0, "gender": 0}
    total = 0
    
    # Thanh tiến trình tqdm
    pbar = tqdm(loader, desc=f"{phase.capitalize()} ", leave=False)
    
    for imgs, labels in pbar:
        # Đẩy dữ liệu sang thiết bị (non_blocking giúp tăng tốc data transfer)
        imgs = imgs.to(device, non_blocking=True)
        labels = {k: v.to(device, non_blocking=True) for k, v in labels.items()}
        
        # Bật tính toán gradient chỉ khi train
        with torch.set_grad_enabled(is_train):
            # Mixed Precision Context (tự động chuyển float32 -> float16 ở những chỗ cần thiết)
            with autocast(enabled=use_cuda):
                outputs = model(imgs)
                loss, _ = model.compute_loss(outputs, labels)
            
            # --- QUÁ TRÌNH LAN TRUYỀN NGƯỢC (BACKPROPAGATION) ---
            if is_train:
                optimizer.zero_grad(set_to_none=True) # Reset gradient sạch sẽ
                
                scaler.scale(loss).backward()         # Scale loss để tránh underflow fp16
                scaler.unscale_(optimizer)            # Unscale trước khi clip gradient
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0) # Chặn bùng nổ gradient
                scaler.step(optimizer)                # Cập nhật trọng số
                scaler.update()                       # Cập nhật scale factor
                
                # Scheduler bước theo batch (nếu dùng CosineAnnealingWarmRestarts)
                if scheduler and isinstance(scheduler, torch.optim.lr_scheduler.CosineAnnealingWarmRestarts):
                    scheduler.step()
            
            # --- TÍNH TOÁN METRICS ---
            batch_size = imgs.size(0)
            total_loss += loss.item() * batch_size
            total += batch_size
            
            # Tính accuracy cho từng task
            for task in outputs:
                pred = outputs[task].argmax(dim=1)
                correct[task] += (pred == labels[task]).sum().item()
            
            # Hiển thị loss trên thanh pbar
            postfix = {"loss": f"{total_loss/total:.4f}"}
            if is_train:
                postfix["lr"] = f"{optimizer.param_groups[0]['lr']:.2e}"
            pbar.set_postfix(postfix)
    
    # Scheduler bước theo epoch (nếu không phải WarmRestarts)
    if is_train and scheduler and not isinstance(scheduler, torch.optim.lr_scheduler.CosineAnnealingWarmRestarts):
        scheduler.step()
    
    # Tổng hợp kết quả
    acc = {f"{task}_acc": v / total for task, v in correct.items()}
    acc["avg_acc"] = np.mean(list(acc.values()))
    
    return {"loss": total_loss / total, **acc}

# ============================================================
# 4. HÀM HUẤN LUYỆN CHÍNH (TRAIN MODEL)
# ============================================================

def train_model(model,
                train_loader,
                val_loader,
                num_epochs=20,         
                checkpoint_path=None,  # Đường dẫn file .pt để resume train
                save_dir="./checkpoints"):
    
    # Tạo thư mục lưu checkpoint
    os.makedirs(save_dir, exist_ok=True)
    set_seed(42)
    model = model.to(device)
    
    # Cấu hình Optimizer & Scheduler
    optimizer = torch.optim.AdamW(model.parameters(), lr=3e-3, weight_decay=1e-2)
    # CosineAnnealingWarmRestarts giúp model thoát khỏi local minima
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=10, T_mult=2, eta_min=1e-5
    )
    
    best_val_loss = float('inf')
    start_epoch = 0
    
    # Khởi tạo lịch sử training
    history = {
        "train_loss": [], "val_loss": [],
        "val_acc_age": [], "val_acc_race": [],
        "val_acc_gender": [], "val_acc_avg": []
    }
    
    # --- LOAD CHECKPOINT (RESUME TRAINING) ---
    if checkpoint_path and os.path.exists(checkpoint_path):
        print(f"Loading checkpoint from: {checkpoint_path}")
        try:
            ckpt = safe_load(checkpoint_path)
            # Load lại toàn bộ trạng thái
            model.load_state_dict(ckpt["model_state_dict"])
            optimizer.load_state_dict(ckpt["optimizer_state_dict"])
            scheduler.load_state_dict(ckpt["scheduler_state_dict"])
            scaler.load_state_dict(ckpt["scaler_state_dict"])
            
            start_epoch = ckpt.get("epoch", 0) + 1
            best_val_loss = ckpt.get("best_val_loss", float('inf'))
            
            if "history" in ckpt:
                history = ckpt["history"]
                print("Loaded existing history from checkpoint.")
            else:
                print("Warning: No 'history' found. Starting fresh logs.")
                
            print(f"Successfully resumed from epoch {start_epoch}")
        except Exception as e:
            print(f"Failed to load checkpoint: {e}")
            start_epoch = 0

    print(f"\nStarting training from epoch {start_epoch + 1} -> {num_epochs}")
    
    # --- TRAINING LOOP ---
    for epoch in range(start_epoch, num_epochs):
        print(f"\n{'='*60}")
        print(f"EPOCH {epoch+1:03d}/{num_epochs} | LR: {optimizer.param_groups[0]['lr']:.2e}")
        print(f"{'='*60}")
        
        # Chạy train và validation
        train_metrics = run_epoch(model, train_loader, optimizer, scheduler, scaler, "train")
        val_metrics   = run_epoch(model, val_loader,    None,      None,      scaler, "val")
        
        # Lưu metrics vào history
        history["train_loss"].append(train_metrics["loss"])
        history["val_loss"].append(val_metrics["loss"])
        history["val_acc_age"].append(val_metrics["age_acc"])
        history["val_acc_race"].append(val_metrics["race_acc"])
        history["val_acc_gender"].append(val_metrics["gender_acc"])
        history["val_acc_avg"].append(val_metrics["avg_acc"])
        
        # In kết quả
        print(f"TRAIN -> Loss: {train_metrics['loss']:.4f}")
        print(f"VAL   -> Loss: {val_metrics['loss']:.4f} | "
              f"Age: {val_metrics['age_acc']:.3f} Race: {val_metrics['race_acc']:.3f} Gen: {val_metrics['gender_acc']:.3f}")

        # Chuẩn bị dictionary để lưu
        save_dict = {
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "scheduler_state_dict": scheduler.state_dict(),
            "scaler_state_dict": scaler.state_dict(),
            "history": history,
            "best_val_loss": best_val_loss
        }

        # Lưu model tốt nhất (Best Model)
        if val_metrics["loss"] < best_val_loss:
            best_val_loss = val_metrics["loss"]
            best_path = os.path.join(save_dir, "best_model.pt")
            save_dict["best_val_loss"] = best_val_loss
            torch.save(save_dict, best_path)
            print(f"NEW BEST MODEL! Saved to {best_path}")

        # Lưu checkpoint mỗi epoch (Latest Model)
        if (epoch + 1) % 1 == 0:
            latest_path = os.path.join(save_dir, f"epoch_{epoch+1}.pt")
            torch.save(save_dict, latest_path)
            print(f"Saved checkpoint to {latest_path}")

    print(f"\nTraining finished! Best val loss: {best_val_loss:.4f}")
    return history

In [ ]:
if __name__ == "__main__":

    model = MultiTaskFaceModel().to(device)

    history=train_model(model,
                train_loader,
                val_loader,
                num_epochs=150,
                checkpoint_path="/kaggle/input/model-gra/pytorch/default/1/best_model.pt")


In [ ]:
ckpt = safe_load("/kaggle/input/model-gra/pytorch/default/1/best_model.pt")
history = ckpt["history"]

# Vẽ biểu đồ bằng biến history
epochs_range = range(1, len(history["train_loss"]) + 1)

plt.figure(figsize=(15, 6))

# Loss Chart
plt.subplot(1, 2, 1)
plt.plot(epochs_range, history["train_loss"], 'b-o', label='Train Loss')
plt.plot(epochs_range, history["val_loss"], 'r-o', label='Val Loss')
plt.title('Training & Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# Accuracy Chart
plt.subplot(1, 2, 2)
plt.plot(epochs_range, history["val_acc_age"], label='Age Acc')
plt.plot(epochs_range, history["val_acc_race"], label='Race Acc')
plt.plot(epochs_range, history["val_acc_gender"], label='Gender Acc')
plt.plot(epochs_range, history["val_acc_avg"], 'k--', linewidth=2, label='Avg Acc')
plt.title('Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Test predict

In [ ]:
# ============================================================
# CẤU HÌNH (CONFIG)
# ============================================================
CKPT_PATH = "/kaggle/input/model-gra/pytorch/default/1/best_model.pt"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMAGE_SIZE = 112

# Map ngược label (Số) → Tên lớp (String)
gender_map_inv = {0: "Male", 1: "Female"}

race_map_inv = {
    0: "White", 1: "Black", 2: "Latino_Hispanic",
    3: "East Asian", 4: "Southeast Asian", 5: "Indian", 6: "Middle Eastern"
}

age_map_inv = {
    0: "0-2", 1: "3-9", 2: "10-19", 3: "20-29",
    4: "30-39", 5: "40-49", 6: "50-59", 7: "60-69", 8: "70+"
}

# ============================================================
# TIỀN XỬ LÝ ẢNH
# ============================================================
val_tf = T.Compose([
    T.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    T.ToTensor(),
    T.Normalize([0.5], [0.5]),
])

def process_image(img_path):
    """
    Đọc ảnh và chuyển thành Tensor cho model.
    """
    try:
        img = Image.open(img_path).convert("RGB")
        return val_tf(img).unsqueeze(0) # [1, C, H, W]
    except Exception as e:
        print(f"Lỗi khi đọc ảnh: {e}")
        return None

# ============================================================
# HÀM DỰ ĐOÁN
# ============================================================
def predict(model, img_tensor):
    model.eval()
    with torch.no_grad():
        img_tensor = img_tensor.to(DEVICE)
        outputs = model(img_tensor)

        pred_gender = outputs["gender"].argmax(dim=1).item()
        pred_race   = outputs["race"].argmax(dim=1).item()
        pred_age    = outputs["age"].argmax(dim=1).item()

    return {
        "gender": gender_map_inv[pred_gender],
        "race":   race_map_inv[pred_race],
        "age":    age_map_inv[pred_age],
    }

# ============================================================
# HÀM HIỂN THỊ ẢNH (MỚI THÊM)
# ============================================================
def visualize_result(img_path, result):
    """
    Hiển thị ảnh gốc và kết quả dự đoán.
    """
    if not os.path.exists(img_path):
        return

    # Mở ảnh gốc để hiển thị (không dùng ảnh đã normalize)
    img = Image.open(img_path).convert("RGB")

    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.axis('off') # Tắt trục tọa độ

    # Tạo tiêu đề chứa kết quả
    title_text = (f"Gender: {result['gender']} | "
                  f"Age: {result['age']}\n"
                  f"Race: {result['race']}")
    
    plt.title(title_text, fontsize=12, color='darkblue', fontweight='bold')
    plt.show()

# ============================================================
# LOAD MODEL & CHẠY
# ============================================================
if __name__ == "__main__":
    # Khởi tạo model
    model = MultiTaskFaceModel()
    
    # Load checkpoint
    if os.path.exists(CKPT_PATH):
        try:
            print(f"Loading checkpoint: {CKPT_PATH}")
            ckpt = torch.load(CKPT_PATH, map_location=DEVICE, weights_only=False)
            model.load_state_dict(ckpt["model_state_dict"], strict=False)
            model.to(DEVICE)
            print("Model loaded successfully!")

            # --- CHẠY THỬ ---
            img_path = "/kaggle/input/fairface/FairFace/val/1000.jpg"
            
            if os.path.exists(img_path):
                # 1. Xử lý ảnh cho model
                img_tensor = process_image(img_path)
                
                if img_tensor is not None:
                    # 2. Dự đoán
                    result = predict(model, img_tensor)
                    
                    # 3. In kết quả ra màn hình console
                    print("\n=== KẾT QUẢ DỰ ĐOÁN ===")
                    print(result)
                    
                    # 4. Hiển thị ảnh minh họa
                    visualize_result(img_path, result)
            else:
                print(f"Không tìm thấy ảnh tại: {img_path}")
                
        except Exception as e:
            print(f"Lỗi khi chạy model: {e}")
    else:
        print(f"Không tìm thấy checkpoint tại: {CKPT_PATH}")

# Dowload best_model.zip

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))
    
download_file('/kaggle/working/checkpoints/best_model.pt', 'best_model') 